In [1]:
from binascii import hexlify
import requests
import json

### Encrypt random text

In [18]:
def encrypt(text):
    url = f"https://aes.cryptohack.org/lazy_cbc/encrypt/{text}/"
    r = requests.get(url)
    # ct = (json.loads(r.text))["ciphertext"]
    return r.text # ct

In [22]:
response = encrypt("abcd123456789012abcd123456789012")#.encode())
response

'{"ciphertext":"94fcd69cf1332823e5a249f58ec4304e"}\n'

In [25]:
cypher = json.loads(response)["ciphertext"]
cypher, cypher*2

('94fcd69cf1332823e5a249f58ec4304e',
 '94fcd69cf1332823e5a249f58ec4304e94fcd69cf1332823e5a249f58ec4304e')

### Check cypher text by doubling the cypher received on the previous step

In [29]:
def receive(cyphertext):
    url = f"https://aes.cryptohack.org/lazy_cbc/receive/{cyphertext}/"
    r = requests.get(url)
    return r.text # ct

In [30]:
receive(cypher*2)

'{"error":"Invalid plaintext: abcd123456789012abcd123456789012d437dd8437c1911a930fbbbd92266dea"}\n'

In [32]:
err_response = 'abcd123456789012abcd123456789012d437dd8437c1911a930fbbbd92266dea'
len(err_response)

64

### Analyse the response: split the response on two blocks and calc the key 

In [37]:
P0 = err_response[:32]
P1 = err_response[32:]
K = int(P0, 16) ^ int(P1, 16) ^ int(cypher, 16)
K_hex = '{:x}'.format(K)
K_hex, len(K_hex)

('eb06192c908a292bdd60e07c4a9acdb6', 32)

### Given the key query a flag

In [39]:
def get_flag(key):
    url = f"https://aes.cryptohack.org/lazy_cbc/get_flag/{key}/"
    r = requests.get(url)
    # ct = (json.loads(r.text))["ciphertext"]
    return r.text # ct

In [41]:
flag_json = get_flag('eb06192c908a292bdd60e07c4a9acdb6')
flag_json

'{"plaintext":"63727970746f7b35306d335f703330706c335f64306e375f3768316e6b5f49565f31355f316d70307237346e375f3f7d"}\n'

### Conver the flag to string

In [53]:
hex_str = json.loads(flag_json)["plaintext"]

res_string = []
for i in range(int(len(hex_str)/2)):
    # c = hex_str[i*2:i*2+2]
    c = chr(int(hex_str[i*2:i*2+2], 16))
    res_string.append(c)

''.join(res_string)

'crypto{50m3_p30pl3_d0n7_7h1nk_IV_15_1mp0r74n7_?}'